In [6]:
import os
import json
import pandas as pd

def collect_data(top_folder, ctx_size):
    """ Collects the minimum test loss and corresponding parameters across all subfolders in the main folder. """
    results = []
    # Traverse through each subfolder in the main folder
    for submain_folder in os.listdir(top_folder):
        main_folder = os.path.join(top_folder, submain_folder)
        for subfolder in os.listdir(main_folder):
            subfolder_path = os.path.join(main_folder, subfolder)
            progress_file = os.path.join(subfolder_path, 'progress.csv')
            params_file = os.path.join(subfolder_path, 'params.json')
            
            # Check if both necessary files exist
            if os.path.exists(progress_file) and os.path.exists(params_file):
                try:
                    # Read progress.csv and find the minimum test loss
                    data = pd.read_csv(progress_file)
                    data.fillna(0, inplace=True)
                    # Read params.json
                    with open(params_file, 'r') as file:
                        params = json.load(file)
                        if ctx_size != None and params['context'] != ctx_size:
                            continue
                        # Collect required params and the corresponding test loss
                        result = {}
                        if 'master_neurons' in params:
                            result['master_neurons'] = params.get('master_neurons')
                        if 'store_embedding' in params:
                            result['store_embedding_neurons'] = params.get('store_embedding')
                        result['learning_rate'] = params.get('learning_rate')

                        result['best_dev_loss'] = data['dev_loss'].min()
                        result['test_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['test_loss'].iloc[0]
                        result['train_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['train_loss'].iloc[0]
                        result['best_test_loss'] = data['test_loss'].min()
                        result['best_train_loss'] = data['train_loss'].min()
                        result['path'] = subfolder_path
                        results.append(result)
                    # if 'test_loss' in data.columns:
                    #     min_loss = data['test_loss'].min()
                    #     # Read params.json
                    #     with open(params_file, 'r') as file:
                    #         params = json.load(file)
                    #         context_size = params.get('context_size')
                    #         # Collect required params and the corresponding test loss
                    #         result = {
                    #             'learning_rate': params.get('learning_rate'),
                    #             'best_test_loss': min_loss
                    #         }
                    #         results.append(result)
                except Exception as e:
                    print(f"Error processing files in {subfolder_path}: {e}")

    return results

def create_results_table(main_folder, ctx_size):
    """ Creates a table of the minimum test losses for each combination of learning_rate, context_size, and samples. """
    data = collect_data(main_folder, ctx_size)
    if data:
        df = pd.DataFrame(data)
        sorted_result_df = df.sort_values(by='best_dev_loss', ascending=True)
    else:
        print("No data collected. Check the contents of your directories.")
    return sorted_result_df

def make_the_result_table(paths, context_sizes):
    # Placeholder for results
    results = []

    # Iterate over each path and context size, call the function, and store the top row
    for num_stores, path in paths.items():
        for context_size in context_sizes:
            df = create_results_table(path, context_size)
            top_row = df.iloc[0]  # Get the top row
            # Extract necessary columns and add custom columns
            result_row = {
                "# of stores": num_stores,
                "context size": context_size,
                "Learning Rate": top_row['learning_rate'],
                "Train Loss": top_row['train_loss(at best_dev)'],
                "Dev Loss": top_row['best_dev_loss'],
                "Test Loss": top_row['test_loss(at best_dev)'],
                # "path": top_row['path']
            }
            results.append(result_row)

    # Combine all top rows into a single DataFrame
    return pd.DataFrame(results)

In [7]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}

df = make_the_result_table(paths, [0, 1, 256])
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             0         0.0100    5.670470  5.657994   5.663223
           3             1         0.0100    5.613812  5.608477   5.609850
           3           256         0.0100    5.616237  5.611364   5.612627
           5             0         0.0010    5.312334  5.292432   5.301958
           5             1         0.0100    5.256996  5.235224   5.247245
           5           256         0.0001    5.249202  5.237397   5.249729
          10             0         0.0100    5.773343  5.808772   5.776743
          10             1         0.0100    5.712263  5.755829   5.720371
          10           256         0.0100    5.731197  5.761904   5.729029
          20             0         0.0010    5.902905  5.892253   5.871639
          20             1         0.0010    5.850531  5.839253   5.818287
          20           256         0.0100    5.857646  5.844117   5.820630
          30             

In [11]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}

df = make_the_result_table(paths, [1])
#df = make_the_result_table(paths, [1, 32, 256])
print(df.to_string(index=False))

No data collected. Check the contents of your directories.


UnboundLocalError: local variable 'sorted_result_df' referenced before assignment

In [5]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/10",
}
df_sym = make_the_result_table(paths, [0, 1, 256])
df_sym.insert(1, 'Architecture Class', "Symmetry_Aware")

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/10",
}
df_van = make_the_result_table(paths, [None])
df_van.insert(1, 'Architecture Class', "Vanilla")

lower_bound = [
    {
                "# of stores": 3,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.19,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 5.75,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.05,
            },
]
df_lower_bound = pd.DataFrame(lower_bound)

df = pd.concat([df_sym, df_van, df_lower_bound])
df.sort_values(by=['# of stores', 'context size'], inplace=True)
print(df.to_string(index=False))

 # of stores Architecture Class context size  Learning Rate  Train Loss   Dev Loss  Test Loss
           3     Symmetry_Aware            0         0.0010   15.696035  15.747026  38.533308
           3     Symmetry_Aware            1         0.0010    6.584759   6.570463   6.565014
           3     Symmetry_Aware          256         0.0100    6.215819   6.195688   6.190809
           3            Vanilla         None         0.0001    6.202766   6.197694   6.193151
           3        Lower bound         None            NaN         NaN        NaN   6.190000
           5     Symmetry_Aware            0         0.0010   14.244384  14.159398  34.947054
           5     Symmetry_Aware            1         0.0100    6.061970   6.045415   6.036085
           5     Symmetry_Aware          256         0.0010    5.769989   5.763230   5.755869
           5            Vanilla         None         0.0001    5.755991   5.759228   5.751869
           5        Lower bound         None            NaN 

In [3]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/10",
}

df = make_the_result_table(paths, [None])
print(df.to_string(index=False))

 # of stores context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3         None         0.0001    6.206678  6.200491   6.195581
           5         None         0.0001    5.755991  5.759228   5.751869
          10         None         0.0001    6.152874  6.159148   6.157146
